In [106]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import pickle

In [92]:
df=pd.read_csv('../input/feb22-may22/bquxjob_68ebf9c1_181a3c02cc5.csv')

In [93]:
df=df.drop(["purchase_15days"],axis=1)

In [94]:
def fillna(data):
    for col in data:
        if data[col].isnull().sum()>=1:
            data[col].fillna(data[col].mode()[0],inplace=True)
    return data 


dct=["total_purchase_3mon","total_screen_view","total_screen_time","user_engagement"]
def out_rem(data):
    for col in dct:
        upper=data[col].quantile(0.98)
        data=data[data[col]<upper]
    return data


a=["Xiaomi","Samsung","Vivo","Realme","OPPO","OnePlus"]
def f_mob(df):
    if df['mobile_brand'] not in a:
        return 'others'
    else:
        return df['mobile_brand']


b=["organic","(none)","share-pathogen","mobile","sms"]
def f_acq(df):
    if df['acquired_medium'] not in b:
        return 'others'
    else:
        return df['acquired_medium']
    
    
def get_dummies(test):
    test= pd.get_dummies(test, columns=['f2r_claimed'])
    test= pd.get_dummies(test, columns=['has_Plantixcredit'])
    test = pd.get_dummies(test, columns=['mobile_brand'])
    test = pd.get_dummies(test, columns=['acquired_medium'])
    test = pd.get_dummies(test, columns=['user_prop_user_language'])
    test.drop(["f2r_claimed_False"], inplace=True, axis=1)
    test.drop(["has_Plantixcredit_False"], inplace=True, axis=1)
    test.drop(["mobile_brand_others"], inplace=True, axis=1)
    test.drop(["acquired_medium_others"], inplace=True, axis=1)
    test.drop(["user_prop_user_language_gu"], inplace=True, axis=1)
    
    return test


def old_customer(data):
    hist_final_p0=data[data["purchase_history_total"]==0]
    hist_final_p1=data[data["purchase_history_total"]>=1]
    hist_final_p1["old_customer"]=1
    hist_final_p0["old_customer"]=0
    test=pd.concat([hist_final_p1,hist_final_p0],axis=0)
    test=test.sample(frac = 1)
    
    return test


def pca(data):
    data=data.reset_index(drop=True)
    pc=data
    pc.drop(["purchase_history_total","total_session_start","old_customer","f2r_claimed_True","has_Plantixcredit_True","total_purchase_3mon","purchase_history_total","acquired_medium_(none)","acquired_medium_mobile","acquired_medium_organic","acquired_medium_share-pathogen","acquired_medium_sms","mobile_brand_OPPO","mobile_brand_OnePlus","mobile_brand_Realme","mobile_brand_Vivo","mobile_brand_Xiaomi","total_screen_time","user_prop_user_language_hi","user_prop_user_language_en","user_prop_user_language_te","user_prop_user_language_mr"],axis=1)

    scalar = StandardScaler()
  
    # fitting
    scalar.fit(pc)
    scaled_data = scalar.transform(pc)
  
    # Let's say, components = 2
    pca = PCA(n_components = 2)
    pca.fit(scaled_data)
    x_pca = pca.transform(scaled_data)
    X_pca=pd.DataFrame(data=x_pca,columns=["events_pca_1","events_pca_2"])
    X_pca.reset_index(drop=True)
    X_data=pd.concat([X_pca,data["f2r_claimed_True"],data["old_customer"],data["total_session_start"],data["has_Plantixcredit_True"],data["total_purchase_3mon"],data["purchase_history_total"],data["acquired_medium_(none)"],data["acquired_medium_mobile"],data["acquired_medium_organic"],data["acquired_medium_share-pathogen"],data["acquired_medium_sms"],data["mobile_brand_OPPO"],data["mobile_brand_OnePlus"],data["mobile_brand_Realme"],data["mobile_brand_Vivo"],data["mobile_brand_Xiaomi"],data["total_screen_time"],data["user_prop_user_language_hi"],data["user_prop_user_language_en"],data["user_prop_user_language_te"],data["user_prop_user_language_mr"]],axis=1)

    return X_data
    

In [95]:
df

In [103]:
def data_preprocessing(data):
    data=data.drop_duplicates(subset='user_id',keep="last")
    data=data.drop(["user_id","first_active_time","last_active_time","user_first_active_time","install_date","device_language","first_order_time","pur_traffic_source_15days","user_id_1","pur_time_15days","Plantixcredit_limit","credit_limit_used","f2r_retailer","open_dukaan","partner_dukaan","region","total_engagement_time","fir_acti_3mon","inorganic","delta","mon_delta","sess_start_sms","sess_start_dyn_link","lst_acti_3mon","total_search_views"],axis=1)
    data["total_screen_time"]=data["total_screen_time"]/60000
    fillna(data)
    data=old_customer(data)
    data=out_rem(data)
    data['acquired_medium'] = data.apply(f_acq, axis='columns')
    data['mobile_brand'] = data.apply(f_mob, axis='columns')
    data=get_dummies(data)
    data=pca(data)
    
    return data

In [104]:
df=data_preprocessing(df)

In [105]:
df

In [ ]:
filename='data_process'
pickle.dump()

In [108]:
### Create a Pickle file using serialization 
pickle_out = open("data_preprocessing.pkl","wb")
pickle.dump(data_preprocessing, pickle_out)
pickle_out.close()